In [1]:
! git clone https://github.com/tangha1004/thesis.git

Cloning into 'thesis'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 92 (delta 59), reused 61 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 330.32 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
%cd '/kaggle/working/thesis'

/kaggle/working/thesis


In [3]:
! git pull origin main

From https://github.com/tangha1004/thesis
 * branch            main       -> FETCH_HEAD
Already up to date.


# **0. Import**

## **0.1. Import basic libraries**

In [4]:
import torch
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd
from IPython.display import display, HTML
!pip install watermark --quiet
import random
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score,
)

%cd '/kaggle/working/thesis/models_VAE'
from models import init_model_dict_multi
from utils import save_model_dict, load_model_dict
from train_test import prepare_trte_data, train_test, train_epoch, test_epoch

/kaggle/working/thesis/models_VAE


In [5]:
import sklearn
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import copy
from sklearn.compose import ColumnTransformer 
from IPython.display import Markdown

import warnings
from sklearn.exceptions import ConvergenceWarning

from scipy.stats import mode
from datetime import datetime

!pip install captum --quiet
from captum.attr import IntegratedGradients, GradientShap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00:00:0100:01


In [6]:
cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'cuda: {cuda}')

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

# Config
rseed = 42
set_seed(rseed)

cuda: True
Random seed set as 42


## **0.2. List hyperparameters**

In [19]:
# no space to pass through cmd
view_list = [1,2,3]
idx_list = [1,2,3]
hidden_dim = [1024,1024,1024]
input_dim = [2000,2000,2000]
latent_space_dim = int((len(view_list) - 1) * 256)
level_2_dim = [1024,1024,1024]
level_3_dim = [512,512,512]
# num_omics x level_2_dim ---> level_4_dim
level_4_dim = int(len(view_list) * 256)
classifier_1_dim = int((len(view_list) - 1) * 128)
class_num = 4
num_class = 4

print_hyper = False
verbose = False
testonly = False

dataset_name = 'tcga-gbm-methxgexcnv-2000-3-omics'
COHORT = 'TCGA_GBM_METHxGExCNV_2000x2000x2000_MinMaxScaler'
data_folder = f'/kaggle/input/{dataset_name}/{COHORT}'
model_folder = '/kaggle/working/models'
train_file = '/kaggle/working/thesis/models_VAE/main.py'

num_epoch = 200
lr = 0.001
batch_size = 32
patience = 23
k_view_list = [1.0,1.0,1.0]
k_kl = 1.0
k_c = 1.0

In [8]:
get_name = COHORT.count('_') + 3

postfix_tr = '_tr'
postfix_te = '_val'

data_folder = f'/kaggle/input/{dataset_name}/{COHORT}'
model_folder = '/kaggle/working/models'
train_file = '/kaggle/working/thesis/models_VAE/main.py'

In [9]:
loc_file_json_id_omic = data_folder + '/1/dct_index_subtype.json'
with open(loc_file_json_id_omic) as file_json_id_omic:
    dct_LABEL_MAPPING_NAME = json.load(file_json_id_omic)
    # dct_LABEL_MAPPING_NAME = {int(k): v for k,v in dct_LABEL_MAPPING_NAME.items()} # convert str number key to int
LABEL_MAPPING_NAME = dct_LABEL_MAPPING_NAME.values()

In [10]:
for fold_id in [3]:
#     print(f'idx data: {fold_id}')
    tmp = list(LABEL_MAPPING_NAME)
    label_files = ['tr', 'te', 'val']
    dict = {
        'tr': 'Train set',
        'te': 'Test set',
        'val': 'Validation set'
    }
    
    print('\nCount per Subtypes: \n')
    for label_file in label_files:
        df = pd.read_csv(f'{data_folder}/{fold_id}/labels_{label_file}.csv', header=None, names=['subtypes'])
        subtype_counts = df['subtypes'].value_counts().sort_index()
        
        print(f'{dict[label_file]}')
        
        res = {}
        for subtype, count in subtype_counts.items():
            res[tmp[subtype]] = count

        print(pd.DataFrame(res, index=[0]).to_string(index=False), '\n')
    
    print('\nCount Samples: \n')
    for idx in view_list:
        res = {}
        for label_file in label_files:
            df = pd.read_csv(f'{data_folder}/{fold_id}/{idx}_{label_file}.csv', header=None)
            res[dict[label_file]] = df.shape[0]
            
        print(pd.DataFrame(res, index=[0]).to_string(index=False), '\n')
    
    print('\nCount Features: \n')
    res={}
    for idx in view_list:
        df = pd.read_csv(f'{data_folder}/{fold_id}/{idx}_featname.csv', header=None, names=['featname'])
        res[f"Omic {idx}"] = df.shape[0]
        
    print(pd.DataFrame(res, index=[0]).to_string(index=False), '\n')


Count per Subtypes: 

Train set
 Classical  Mesenchymal  Neural  Proneural
        43           48      28         43 

Test set
 Classical  Mesenchymal  Neural  Proneural
        14           16       9         15 

Validation set
 Classical  Mesenchymal  Neural  Proneural
        14           17       9         14 


Count Samples: 

 Train set  Test set  Validation set
       162        54              54 

 Train set  Test set  Validation set
       162        54              54 

 Train set  Test set  Validation set
       162        54              54 


Count Features: 

 Omic 1  Omic 2  Omic 3
   2000    2000    2000 



In [11]:
added_softmax = False

def preprocessing_data(tup_tensor_test_data, data_folder):
    data_tr_list = []
    data_te_list = []
    
    for i in view_list:
        data_tr_list.append(torch.tensor(np.loadtxt(os.path.join(data_folder, str(i)+"_tr.csv"), delimiter=','),dtype=torch.float32))
        data_te_list.append(tup_tensor_test_data[i-1])
        if cuda:
            data_tr_list[i-1] = data_tr_list[i-1].to(device)
            data_te_list[i-1] = data_te_list[i-1].to(device)       

    num_tr = data_tr_list[0].shape[0]
    num_te = data_te_list[0].shape[0]
    trte_idx = {}
    trte_idx["tr"] = list(range(num_tr))
    trte_idx["te"] = list(range(num_tr, (num_tr+num_te)))

    num_view = len(view_list)
    data_tensor_list = []
    for i in range(num_view):
        data_tensor_list.append(torch.cat((data_tr_list[i], data_te_list[i]), axis=0))
        if cuda:
            data_tensor_list[i] = data_tensor_list[i].to(device)#cuda()
    
    data_train_list = []
    data_trte_list = []
    for i in range(len(data_tensor_list)):
        data_train_list.append(data_tensor_list[i][trte_idx["tr"]].clone())

        tup_seq_data = (data_tensor_list[i][trte_idx["tr"]].clone(), data_tensor_list[i][trte_idx["te"]].clone())
        data_trte_list.append(
            torch.cat(tup_seq_data,axis=0)
        )
    return data_train_list, data_trte_list,trte_idx

## **0.3. Feature importance**

In [21]:
def custom_logit_predictor(*tup_tensor_data, data_folder):
    global added_softmax
    added_softmax = True

    if cuda:
        model_dict['VAE_multi'].to(device)
    model_dict['VAE_multi'].eval()

    tup_tensor_data = tuple(tensor_data.to(device) if cuda else tensor_data 
                           for tensor_data in tup_tensor_data)

    data_tr_list, data_trte_list, trte_idx = preprocessing_data(tup_tensor_data, data_folder)
    with torch.set_grad_enabled(True):
        predictions = model_dict['VAE_multi'].infer(data_trte_list)
    predictions = predictions[trte_idx["te"],:]
    if added_softmax:
        predictions = F.softmax(predictions, dim=1)

    return predictions

model_dict=None
def load_model(data_folder, model_folder):
    data_tr_list, data_trte_list, trte_idx, labels_trte = prepare_trte_data(data_folder, view_list, postfix_tr='_tr', postfix_te='_val')
    dim_list = [x.shape[1] for x in data_tr_list]

    global model_dict
    model_dict = init_model_dict_multi(view_list,
                    input_dim, latent_space_dim, 
                    level_2_dim, level_3_dim,
                    level_4_dim, 
                    classifier_1_dim, class_num)
    print(model_folder)
    model_dict = load_model_dict(model_folder, model_dict)

## **0.4. Metrics**

In [13]:
def display_classification_report(
    n_class,
    conf_matrix,
    avg_report,
    label_mapping_name,
    cmap="Blues",
    fmt=".2%",
    annot=True,
    path=None,  # str path to save fig. If not None
    shown=True,
):
    clf_df = avg_report
    clf_df.loc[["precision", "recall"], "accuracy"] = np.nan

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figwidth(12)
    
    ConfusionMatrixDisplay(conf_matrix, display_labels=label_mapping_name).plot(cmap=cmap, ax=ax1)
    
    sns.heatmap(clf_df.iloc[:-1, :].T, annot=annot, cmap=cmap, robust=True, ax=ax2, fmt=fmt)
    
    if path is not None:
        fig.savefig(path, dpi=300)
    if shown:
        plt.show()
    else:
        plt.close(fig)

In [14]:
def calculate_average_report(reports):
    """Calculate the average classification report from a list of reports."""
    avg_report = pd.DataFrame(reports[0]).copy()
    for report in reports[1:]:
        avg_report += pd.DataFrame(report)
    avg_report /= len(reports)
    return avg_report

def evaluate_model(bool_report=True, _type_data='te'):
    conf_matrix = np.zeros((num_class, num_class), dtype=int)

    reports = []
    results = []

    for idx in idx_list:
        cur_model_folder = f'{model_folder}/{idx}'
        cur_data_folder = f"{data_folder}/{idx}/"
        load_model(cur_data_folder, cur_model_folder)

        _data_list = []

        _label = np.loadtxt(os.path.join(cur_data_folder, f"labels_{_type_data}.csv"), delimiter=',').astype(int)

        for i in view_list:
            _data_loc = os.path.join(cur_data_folder, f"{i}_{_type_data}.csv")
            _data_list.append(np.loadtxt(_data_loc, delimiter=','))
        
        _tensor_data_list = tuple(torch.tensor(np_arr, dtype=torch.float32).to(device) for np_arr in _data_list)
        pred = custom_logit_predictor(*_tensor_data_list, data_folder=cur_data_folder)
        pred = np.array(torch.argmax(pred.cpu(), dim=1))

        fold_conf_matrix = confusion_matrix(_label, pred, labels=np.arange(num_class))
        conf_matrix += fold_conf_matrix
        
        acc = accuracy_score(_label, pred)
        f1_macro = f1_score(_label, pred, average='macro')
        f1_weighted = f1_score(_label, pred, average='weighted')
        results.append((idx, acc, f1_macro, f1_weighted))

        # Get classification report for the current fold
        report = classification_report(_label, pred, target_names=LABEL_MAPPING_NAME, output_dict=True)
        reports.append(report)
        
        
    avg_report = calculate_average_report(reports)

    if bool_report:
        if not os.path.exists("/kaggle/working/phase1"):
            os.makedirs("/kaggle/working/phase1")
        
        # Calculate average classification report        
        # Calculate the average of the models
        df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'F1 Macro', 'F1 Weighted'])
        avg_row = ['Average', df['Accuracy'].mean(), df['F1 Macro'].mean(), df['F1 Weighted'].mean()]
        df.loc[len(df)] = avg_row

        print(df.to_string(index=False))

        # Display confusion matrix and classification report
        display_classification_report(
            num_class,
            conf_matrix,
            avg_report,
            LABEL_MAPPING_NAME,
            path=f"/kaggle/working/phase1/Evaluate_model_{_type_data}",
        )

    return avg_report['macro avg']['f1-score']

In [15]:
retrain = True
if retrain:
    model_folder = '/kaggle/working/models'
    saved_model_dict_folder = model_folder
    fold_list = [1,2,3,4]
    for fold_id in fold_list:
        print(f'idx data: {fold_id}')

        data_folder_idx = f'{data_folder}/{fold_id}'
        model_folder_idx = f'{model_folder}/{fold_id}'
        
        
        # Run main.py with all required arguments
        !python '{train_file}' \
            '{view_list}' '{hidden_dim}' '{input_dim}' '{latent_space_dim}' \
            '{level_2_dim}' '{level_3_dim}' '{level_4_dim}' '{classifier_1_dim}' \
            '{class_num}' '{print_hyper}' '{verbose}' '{testonly}' \
            '{data_folder_idx}' '{model_folder_idx}' '{saved_model_dict_folder}' \
            '{num_epoch}' '{lr}' '{batch_size}' '{patience}' \
            '{k_view_list}' '{k_kl}' '{k_c}'
        
        print('*'*100)
else:
    model_folder = f'/kaggle/input/{dataset_name}/models'

idx data: 1
cuda: True
Random seed set as 42

Training...
Early stop at epoch 35th after 23 epochs not increasing score from epoch 12th with best score 0.8169450496247882
Early stopping triggered. Using best model from epoch 12
****************************************************************************************************
idx data: 2
cuda: True
Random seed set as 42

Training...
Early stop at epoch 59th after 23 epochs not increasing score from epoch 36th with best score 0.8889506859473489
Early stopping triggered. Using best model from epoch 36
****************************************************************************************************
idx data: 3
cuda: True
Random seed set as 42

Training...
Early stop at epoch 48th after 23 epochs not increasing score from epoch 25th with best score 0.8110668084352294
Early stopping triggered. Using best model from epoch 25
****************************************************************************************************
idx data: 4
c

In [22]:
evaluate_model(_type_data='tr')
evaluate_model(_type_data='val')
evaluate_model(_type_data='te')

/kaggle/working/models/1


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL models.VAE_multi was not an allowed global by default. Please use `torch.serialization.add_safe_globals([VAE_multi])` or the `torch.serialization.safe_globals([VAE_multi])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_onl